# Multiple dispatch

In this notebook we'll explore **multiple dispatch**, which is a key feature of Julia.

Multiple dispatch makes software *generic* and *fast*!

#### Starting with the familiar

To understand multiple dispatch in Julia, let's start with what we've already seen.

We can declare functions in Julia without giving Julia any information about the types of the input arguments that function will receive:

In [1]:
f(x) = x^2

f (generic function with 1 method)

and then Julia will determine on its own which input argument types make sense and which do not:

In [2]:
@show f(10)        # makes sense
@show f([1, 2, 3]) # doesn't make sense

f(10) = 100


MethodError: MethodError: no method matching ^(::Array{Int64,1}, ::Int64)
Closest candidates are:
  ^(!Matched::Float16, ::Integer) at math.jl:795
  ^(!Matched::Missing, ::Integer) at missing.jl:124
  ^(!Matched::Missing, ::Number) at missing.jl:97
  ...

## Specifying argument types
Let's write a function `foo` that only takes strings as inputs.

In [4]:
foo(x::String, y::String) = println("My inputs x and y are both strings!")

foo (generic function with 1 method)

In [5]:
foo("hello", "hi!")  # works!
foo(3, 4)            # Doesn't work, we never defined for Integers

My inputs x and y are both strings!


MethodError: MethodError: no method matching foo(::Int64, ::Int64)

In [6]:
foo(x::Int, y::Int) = println("My inputs x and y are both integers!")

foo (generic function with 2 methods)

In [9]:
@show foo(3, 4)
@show foo("hello", "hi!")

My inputs x and y are both integers!
foo(3, 4) = nothing
My inputs x and y are both strings!
foo("hello", "hi!") = nothing


This is starting to get to the heart of multiple dispatch. When we declared

```julia
foo(x::Int, y::Int) = println("My inputs x and y are both integers!")
```
we didn't overwrite or replace
```julia
foo(y::String, y::String)```

Instead, we just added an additional ***method*** to the ***generic function*** called `foo`.

A ***generic function*** is the abstract concept associated with a particular operation.

For example, the generic function `+` represents the concept of addition.

A ***method*** is a specific implementation of a generic function for *particular argument types*.

For example, `+` has methods that accept floating point numbers, integers, matrices, etc.

We can use the `methods` to see how many methods there are for `foo`.

In [10]:
methods(foo)

# 2 methods for generic function "foo":
[1] foo(x::Int64, y::Int64) in Main at In[6]:1
[2] foo(x::String, y::String) in Main at In[4]:1

Aside: how many methods do you think there are for addition?

In [11]:
methods(+)

# 163 methods for generic function "+":
[1] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:277
[2] +(x::Bool, y::Bool) in Base at bool.jl:104
[3] +(x::Bool) in Base at bool.jl:101
[4] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:112
[5] +(x::Bool, z::Complex) in Base at complex.jl:284
[6] +(a::Float16, b::Float16) in Base at float.jl:392
[7] +(x::Float32, y::Float32) in Base at float.jl:394
[8] +(x::Float64, y::Float64) in Base at float.jl:395
[9] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:278
[10] +(z::Complex{Bool}, x::Real) in Base at complex.jl:292
[11] +(::Missing, ::Missing) in Base at missing.jl:96
[12] +(::Missing) in Base at missing.jl:83
[13] +(::Missing, ::Number) in Base at missing.jl:97
[14] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:106
[15] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:406
[16] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:434
[17] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:433
[18] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:432
[19] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:403
[20] +(x::BigInt, c::Union{UInt16, UInt32, UInt8}) in Base.GMP at gmp.jl:440
[21] +(x::BigInt, c::Union{Int16, Int32, Int8}) in Base.GMP at gmp.jl:446
[22] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:556
[23] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:549
[24] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:543
[25] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:402
[26] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:371
[27] +(x::BigFloat, c::Union{UInt16, UInt32, UInt8}) in Base.MPFR at mpfr.jl:378
[28] +(x::BigFloat, c::Union{Int16, Int32, Int8}) in Base.MPFR at mpfr.jl:386
[29] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:394
[30] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\periods.jl:343
[31] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\periods.jl:341
[32] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\periods.jl:371
[33] +(x::Dates.Date, y::Dates.Day) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:74
[34] +(x::Dates.Date, y::Dates.Week) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:72
[35] +(dt::Dates.Date, z::Dates.Month) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:55
[36] +(dt::Dates.Date, y::Dates.Year) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:28
[37] +(dt::Dates.Date, t::Dates.Time) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:19
[38] +(t::Dates.Time, dt::Dates.Date) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:20
[39] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:78
[40] +(dt::Dates.DateTime, z::Dates.Month) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:48
[41] +(dt::Dates.DateTime, y::Dates.Year) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\s

So, we now can call `foo` on integers or strings. When you call `foo` on a particular set of arguments, Julia will infer the types of the inputs and dispatch the appropriate method. *This* is multiple dispatch.

Multiple dispatch makes our code generic and fast. Our code can be generic and flexible because we can write code in terms of abstract operations such as addition and multiplication, rather than in terms of specific implementations. At the same time, our code runs quickly because Julia is able to call efficient methods for the relevant types.

To see which method is being dispatched when we call a generic function, we can use the @which macro:

In [12]:
@which foo(3, 4)

foo(x::Int64, y::Int64) in Main at In[6]:1

Let's see what happens when we use `@which` with the addition operator!

In [13]:
@which 3.0 + 3.0

+(x::Float64, y::Float64) in Base at float.jl:395

### We can also dispatch using an ***abstract type*** 

In [18]:
foo(x::Number, y::Number) = println("My inputs x and y are both numbers!")
foo(3.0, 4.0)

My inputs x and y are both numbers!


We can also add a fallback, duck-typed method for `foo` that takes inputs of any type:

In [2]:
foo(x, y) = println("I accept inputs of any type!")   #This is the same as foo(x::Any,y::Any)

foo (generic function with 1 method)

Given the methods we've already written for `foo` so far, this method will be called whenever we pass non-numbers to `foo`:

In [3]:
v = rand(3)
foo(v, v)

I accept inputs of any type!


# Types

## An Abstract Type 
This can be inherited by a concrete type to provide fall-back behavior

In [4]:
abstract type Animal end 

## Concrete Types (Composite Types)
These are Julia's primary objects and can inherit from abstract types

In [5]:
struct Dog <: Animal 
    name::Symbol 
    bark::String
end

struct Cat <: Animal 
    name::Symbol 
    meow::String 
end 

## Constructors

In [6]:
my_dog = Dog(:Cash,"woof!!!")
my_cat = Cat(:Fluffy,"meow...")

Cat(:Fluffy, "meow...")

## Dispatch on Animal Type

In [9]:
function speak(dog::Dog)
    println("Animal $(dog.name) says $(dog.bark)")
end
speak(cat::Cat) = println("Animal $(cat.name) says $(cat.meow)")

speak(my_dog)
speak(my_cat)

Animal Cash says woof!!!
Animal Fluffy says meow...


## Fallback to Abstract Method

In [11]:
shake_paw(animal::Animal) = println("Animal $(animal.name) shakes paws")
shake_paw(my_dog)
shake_paw(my_cat)

Animal Cash shakes paws
Animal Fluffy shakes paws


# One Hot Vector

Often used in machine learning, a "one hot" vector is a vector of all zeros, except for a single `1` entry.
Representing it as a standard vector is memory-inefficient, so it cries out for a special implementation.

## AbstractVector is a Base Julia Abstract Type

In [2]:
struct OneHotVector <: AbstractVector{Int}  #This is a parametric type
    idx::Int
    len::Int
end

## Add new dispatch methods for OneHotVector

In [3]:
Base.size(v::OneHotVector) = (v.len,)
Base.getindex(v::OneHotVector, i::Integer) = Int(i == v.idx)

In [4]:
OneHotVector(3, 10)

10-element OneHotVector:
 0
 0
 1
 0
 0
 0
 0
 0
 0
 0

In [16]:
A = rand(5,5)

5×5 Array{Float64,2}:
 0.650364  0.73805   0.332411   0.489939  0.938504 
 0.616106  0.679628  0.808391   0.421254  0.867379 
 0.777461  0.605425  0.178469   0.882039  0.0739717
 0.336199  0.913348  0.0821768  0.896052  0.735438 
 0.910351  0.685391  0.292147   0.939484  0.0578411

## Multiplication will just work

In [17]:
A * OneHotVector(3, 5)

5-element Array{Float64,1}:
 0.332411415951795  
 0.8083911312365872 
 0.17846920911730013
 0.08217683310139967
 0.2921466198623819 

## Convert to a Julia Vector (works since we defined size and getindex)

In [18]:
Vector(OneHotVector(3,5))

5-element Array{Int64,1}:
 0
 0
 1
 0
 0